# **Code de reproduction de l'article :** How transferable are features in deep neural networks?

In [1]:
%load_ext autoreload
%autoreload 2
import os
import json
from tqdm import tqdm
import torch 
import torch.nn as nn
import models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import Adam
print(f"device avaible : {'cuda' if torch.cuda.is_available() else 'cpu'}")

device avaible : cuda


# **Dataset**

Création de deux datasets distincts : 
- A contient les chiffres de 0 à 4
- B contient les chiffres de 5 à 9 

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train.values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0 
X_test = torch.tensor(X_test.values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# création des datasets A (0-4) et B (5-9)
def create_subset(X, y, class_range):
    indices = [i for i, label in enumerate(y) if label in class_range]
    return Subset(torch.utils.data.TensorDataset(X, y), indices)

train_set_A = create_subset(X_train, y_train, range(0, 5))
train_set_B = create_subset(X_train, y_train, range(5, 10))

test_set_A = create_subset(X_test, y_test, range(0, 5))
test_set_B = create_subset(X_test, y_test, range(5, 10))


train_loader_A = DataLoader(train_set_A, batch_size=64, shuffle=True)
train_loader_B = DataLoader(train_set_B, batch_size=64, shuffle=True)

test_loader_A = DataLoader(test_set_A, batch_size=64, shuffle=False)
test_loader_B = DataLoader(test_set_B, batch_size=64, shuffle=False)

# **Entraînement des modèles de bases**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
path = "./resultats/"

resA  = []
resB = []
print("Entrainement du modèle A")
print()
for _ in range(10): 
    modelA = models.CNN()
    optimizerA = Adam(modelA.parameters())
    modelA.train_model(train_loader_A,test_loader_A, loss_fn, n_epochs=2, optimizer=optimizerA)
    resA.append(modelA.test(test_loader_A))
torch.save(modelA, "./models/baseA.torch")

print()
print("Entrainement du modèle B")
print()
for _ in range(10): 
    modelB = models.CNN()
    optimizerB= Adam(modelB.parameters())
    modelB.train_model(train_loader_B,test_loader_B, loss_fn, n_epochs=2, optimizer=optimizerB)
    resB.append(modelB.test(test_loader_B))
torch.save(modelB, "./models/baseB.torch")

with open(os.path.join(path, "base.json"), 'a') as file:
        json.dump({"model A" : resA}, file)
        json.dump({"model B" : resB}, file)

# **Réentrainement des modèles**

Nous réalisons l'expérience décrite dans le papier. Nous prenons le modèle B comme référence. Nous allons faire une boucle pour itérer sur le nombre de couche du réseau. Commençons pas le cas ou les paramètres des couches copiées sont gelés. Les paramètres du classifieur sont toujours réappris

## modèles BNB

In [ ]:
path = "./resultats/"


basemodel = torch.load("./models/baseB.torch")
loss_fn = nn.CrossEntropyLoss()



for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.conv[i] = basemodel.conv[i]
        for param in newmodel.conv[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    with open(os.path.join(path, "BNB.json"), 'a') as file:
        json.dump(res, file)

## Modèles AnB

In [5]:
path = "./resultats/"


basemodel = torch.load("./models/baseA.torch")
loss_fn = nn.CrossEntropyLoss()


for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.conv[i] = basemodel.conv[i]
        for param in newmodel.conv[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    with open(os.path.join(path, "ANB.json"), 'a') as file:
        json.dump(res, file)

/tmp/ipykernel_3975/3633339083.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it/

Epoch 1, Loss: 0.5182
Test set: Avg. loss: 0.1057, Accuracy: 6734/6949 (96.91%)
Epoch 2, Loss: 0.0815
Test set: Avg. loss: 0.0530, Accuracy: 6838/6949 (98.40%)
Test set: Avg. loss: 0.0530, Accuracy: 6838/6949 (98.40%)
Epoch 1, Loss: 0.9466
Test set: Avg. loss: 0.0919, Accuracy: 6752/6949 (97.17%)
Epoch 2, Loss: 0.0744
Test set: Avg. loss: 0.0495, Accuracy: 6838/6949 (98.40%)
Test set: Avg. loss: 0.0495, Accuracy: 6838/6949 (98.40%)
Epoch 1, Loss: 0.9546
Test set: Avg. loss: 0.0752, Accuracy: 6800/6949 (97.86%)
Epoch 2, Loss: 0.0717
Test set: Avg. loss: 0.0402, Accuracy: 6866/6949 (98.81%)
Test set: Avg. loss: 0.0402, Accuracy: 6866/6949 (98.81%)
Epoch 1, Loss: 1.2285
Test set: Avg. loss: 0.1254, Accuracy: 6659/6949 (95.83%)
Epoch 2, Loss: 0.0870
Test set: Avg. loss: 0.0468, Accuracy: 6847/6949 (98.53%)
Test set: Avg. loss: 0.0468, Accuracy: 6847/6949 (98.53%)
Epoch 1, Loss: 0.4414
Test set: Avg. loss: 0.1121, Accuracy: 6703/6949 (96.46%)
Epoch 2, Loss: 0.0601
Test set: Avg. loss: 0.041

Etape :  12%|█▎        | 1/8 [00:36<04:13, 36.20s/it]

Test set: Avg. loss: 0.0313, Accuracy: 6877/6949 (98.96%)
Epoch 1, Loss: 0.8638
Test set: Avg. loss: 0.0998, Accuracy: 6736/6949 (96.93%)
Epoch 2, Loss: 0.0771
Test set: Avg. loss: 0.0488, Accuracy: 6833/6949 (98.33%)
Test set: Avg. loss: 0.0488, Accuracy: 6833/6949 (98.33%)
Epoch 1, Loss: 0.4592
Test set: Avg. loss: 0.0761, Accuracy: 6785/6949 (97.64%)
Epoch 2, Loss: 0.0659
Test set: Avg. loss: 0.0488, Accuracy: 6847/6949 (98.53%)
Test set: Avg. loss: 0.0488, Accuracy: 6847/6949 (98.53%)
Epoch 1, Loss: 0.7986
Test set: Avg. loss: 0.1009, Accuracy: 6729/6949 (96.83%)
Epoch 2, Loss: 0.0757
Test set: Avg. loss: 0.0534, Accuracy: 6833/6949 (98.33%)
Test set: Avg. loss: 0.0534, Accuracy: 6833/6949 (98.33%)
Epoch 1, Loss: 0.4755
Test set: Avg. loss: 0.0668, Accuracy: 6806/6949 (97.94%)
Epoch 2, Loss: 0.0709
Test set: Avg. loss: 0.0562, Accuracy: 6835/6949 (98.36%)
Test set: Avg. loss: 0.0562, Accuracy: 6835/6949 (98.36%)
Epoch 1, Loss: 0.7090
Test set: Avg. loss: 0.0805, Accuracy: 6774/6949

Etape :  25%|██▌       | 2/8 [01:12<03:37, 36.19s/it]

...

Test set: Avg. loss: 2.0136, Accuracy: 3777/6949 (54.35%)
Epoch 1, Loss: 0.3358
Test set: Avg. loss: 0.0727, Accuracy: 6786/6949 (97.65%)
Epoch 2, Loss: 0.0774
Test set: Avg. loss: 0.0547, Accuracy: 6833/6949 (98.33%)
Test set: Avg. loss: 0.0547, Accuracy: 6833/6949 (98.33%)
Epoch 1, Loss: 0.3303
Test set: Avg. loss: 0.0835, Accuracy: 6759/6949 (97.27%)
Epoch 2, Loss: 0.0734
Test set: Avg. loss: 0.0524, Accuracy: 6832/6949 (98.32%)
Test set: Avg. loss: 0.0524, Accuracy: 6832/6949 (98.32%)
Epoch 1, Loss: 0.2948
Test set: Avg. loss: 0.0977, Accuracy: 6727/6949 (96.81%)
Epoch 2, Loss: 0.0805
Test set: Avg. loss: 0.0633, Accuracy: 6814/6949 (98.06%)
Test set: Avg. loss: 0.0633, Accuracy: 6814/6949 (98.06%)
Epoch 1, Loss: 0.2660
Test set: Avg. loss: 0.0830, Accuracy: 6772/6949 (97.45%)
Epoch 2, Loss: 0.0851
Test set: Avg. loss: 0.0594, Accuracy: 6820/6949 (98.14%)
Test set: Avg. loss: 0.0594, Accuracy: 6820/6949 (98.14%)
Epoch 1, Loss: 0.4174
Test set: Avg. loss: 0.0923, Accuracy: 6755/6949

Etape :  88%|████████▊ | 7/8 [03:56<00:32, 32.67s/it]

Test set: Avg. loss: 0.0600, Accuracy: 6814/6949 (98.06%)
Epoch 1, Loss: 0.4281
Test set: Avg. loss: 0.1647, Accuracy: 6549/6949 (94.24%)
Epoch 2, Loss: 0.1288
Test set: Avg. loss: 0.0929, Accuracy: 6733/6949 (96.89%)
Test set: Avg. loss: 0.0929, Accuracy: 6733/6949 (96.89%)
Epoch 1, Loss: 0.2913
Test set: Avg. loss: 0.0703, Accuracy: 6792/6949 (97.74%)
Epoch 2, Loss: 0.0655
Test set: Avg. loss: 0.0483, Accuracy: 6842/6949 (98.46%)
Test set: Avg. loss: 0.0483, Accuracy: 6842/6949 (98.46%)
Epoch 1, Loss: 0.4061
Test set: Avg. loss: 0.1106, Accuracy: 6694/6949 (96.33%)
Epoch 2, Loss: 0.1281
Test set: Avg. loss: 0.0727, Accuracy: 6787/6949 (97.67%)
Test set: Avg. loss: 0.0727, Accuracy: 6787/6949 (97.67%)
Epoch 1, Loss: 0.3638
Test set: Avg. loss: 0.1099, Accuracy: 6721/6949 (96.72%)
Epoch 2, Loss: 0.1054
Test set: Avg. loss: 0.0680, Accuracy: 6784/6949 (97.63%)
Test set: Avg. loss: 0.0680, Accuracy: 6784/6949 (97.63%)
Epoch 1, Loss: 0.2668
Test set: Avg. loss: 0.0803, Accuracy: 6771/6949

Etape : 100%|██████████| 8/8 [04:26<00:00, 33.31s/it]

Test set: Avg. loss: 0.0683, Accuracy: 6783/6949 (97.61%)


Cette fois-ci on copie des paramètres mais on ne les gèles pas.

## Modèles BnB+

In [3]:
path = "./resultats/"


basemodel = torch.load("./models/baseB.torch")
loss_fn = nn.CrossEntropyLoss()



for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.conv[i] = basemodel.conv[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    with open(os.path.join(path, "BNB+.json"), 'a') as file:
        json.dump(res, file)

/tmp/ipykernel_3975/2828188953.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseB.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it/

Epoch 1, Loss: 0.7028
Test set: Avg. loss: 0.0892, Accuracy: 6750/6949 (97.14%)
Epoch 2, Loss: 0.0652
Test set: Avg. loss: 0.0502, Accuracy: 6835/6949 (98.36%)
Test set: Avg. loss: 0.0502, Accuracy: 6835/6949 (98.36%)
Epoch 1, Loss: 0.8152
Test set: Avg. loss: 0.1058, Accuracy: 6730/6949 (96.85%)
Epoch 2, Loss: 0.0789
Test set: Avg. loss: 0.0525, Accuracy: 6839/6949 (98.42%)
Test set: Avg. loss: 0.0525, Accuracy: 6839/6949 (98.42%)
Epoch 1, Loss: 0.3367
Test set: Avg. loss: 0.0662, Accuracy: 6806/6949 (97.94%)
Epoch 2, Loss: 0.0589
Test set: Avg. loss: 0.0404, Accuracy: 6860/6949 (98.72%)
Test set: Avg. loss: 0.0404, Accuracy: 6860/6949 (98.72%)
Epoch 1, Loss: 0.5374
Test set: Avg. loss: 0.0825, Accuracy: 6773/6949 (97.47%)
Epoch 2, Loss: 0.0792
Test set: Avg. loss: 0.0543, Accuracy: 6831/6949 (98.30%)
Test set: Avg. loss: 0.0543, Accuracy: 6831/6949 (98.30%)
Epoch 1, Loss: 0.8216
Test set: Avg. loss: 0.0669, Accuracy: 6791/6949 (97.73%)
Epoch 2, Loss: 0.0734
Test set: Avg. loss: 0.042

Etape :  12%|█▎        | 1/8 [00:39<04:36, 39.52s/it]

Test set: Avg. loss: 0.0517, Accuracy: 6834/6949 (98.35%)
Epoch 1, Loss: 0.6352
Test set: Avg. loss: 0.0762, Accuracy: 6789/6949 (97.70%)
Epoch 2, Loss: 0.0730
Test set: Avg. loss: 0.0486, Accuracy: 6838/6949 (98.40%)
Test set: Avg. loss: 0.0486, Accuracy: 6838/6949 (98.40%)
Epoch 1, Loss: 0.7616
Test set: Avg. loss: 0.0888, Accuracy: 6756/6949 (97.22%)
Epoch 2, Loss: 0.0832
Test set: Avg. loss: 0.0496, Accuracy: 6846/6949 (98.52%)
Test set: Avg. loss: 0.0496, Accuracy: 6846/6949 (98.52%)
Epoch 1, Loss: 0.7379
Test set: Avg. loss: 0.1025, Accuracy: 6725/6949 (96.78%)
Epoch 2, Loss: 0.0954
Test set: Avg. loss: 0.0635, Accuracy: 6815/6949 (98.07%)
Test set: Avg. loss: 0.0635, Accuracy: 6815/6949 (98.07%)
Epoch 1, Loss: 0.7300
Test set: Avg. loss: 0.0843, Accuracy: 6759/6949 (97.27%)
Epoch 2, Loss: 0.0771
Test set: Avg. loss: 0.0511, Accuracy: 6849/6949 (98.56%)
Test set: Avg. loss: 0.0511, Accuracy: 6849/6949 (98.56%)
Epoch 1, Loss: 0.5275
Test set: Avg. loss: 0.0758, Accuracy: 6789/6949

Etape :  25%|██▌       | 2/8 [01:18<03:53, 38.91s/it]

...

Test set: Avg. loss: 0.0361, Accuracy: 6874/6949 (98.92%)
Epoch 1, Loss: 0.2216
Test set: Avg. loss: 0.0531, Accuracy: 6821/6949 (98.16%)
Epoch 2, Loss: 0.0538
Test set: Avg. loss: 0.0336, Accuracy: 6876/6949 (98.95%)
Test set: Avg. loss: 0.0336, Accuracy: 6876/6949 (98.95%)
Epoch 1, Loss: 0.1997
Test set: Avg. loss: 0.0550, Accuracy: 6824/6949 (98.20%)
Epoch 2, Loss: 0.0522
Test set: Avg. loss: 0.0381, Accuracy: 6857/6949 (98.68%)
Test set: Avg. loss: 0.0381, Accuracy: 6857/6949 (98.68%)
Epoch 1, Loss: 0.1837
Test set: Avg. loss: 0.0493, Accuracy: 6852/6949 (98.60%)
Epoch 2, Loss: 0.0476
Test set: Avg. loss: 0.0352, Accuracy: 6884/6949 (99.06%)
Test set: Avg. loss: 0.0352, Accuracy: 6884/6949 (99.06%)
Epoch 1, Loss: 0.2088
Test set: Avg. loss: 0.0688, Accuracy: 6799/6949 (97.84%)
Epoch 2, Loss: 0.0488
Test set: Avg. loss: 0.0340, Accuracy: 6874/6949 (98.92%)
Test set: Avg. loss: 0.0340, Accuracy: 6874/6949 (98.92%)
Epoch 1, Loss: 0.1890
Test set: Avg. loss: 0.0589, Accuracy: 6823/6949

Etape :  88%|████████▊ | 7/8 [04:30<00:38, 38.70s/it]

Test set: Avg. loss: 0.0348, Accuracy: 6875/6949 (98.94%)
Epoch 1, Loss: 0.2027
Test set: Avg. loss: 0.0456, Accuracy: 6854/6949 (98.63%)
Epoch 2, Loss: 0.0533
Test set: Avg. loss: 0.0362, Accuracy: 6873/6949 (98.91%)
Test set: Avg. loss: 0.0362, Accuracy: 6873/6949 (98.91%)
Epoch 1, Loss: 0.2123
Test set: Avg. loss: 0.0583, Accuracy: 6829/6949 (98.27%)
Epoch 2, Loss: 0.0608
Test set: Avg. loss: 0.0506, Accuracy: 6844/6949 (98.49%)
Test set: Avg. loss: 0.0506, Accuracy: 6844/6949 (98.49%)
Epoch 1, Loss: 0.1767
Test set: Avg. loss: 0.0578, Accuracy: 6815/6949 (98.07%)
Epoch 2, Loss: 0.0499
Test set: Avg. loss: 0.0512, Accuracy: 6842/6949 (98.46%)
Test set: Avg. loss: 0.0512, Accuracy: 6842/6949 (98.46%)
Epoch 1, Loss: 0.1658
Test set: Avg. loss: 0.0510, Accuracy: 6842/6949 (98.46%)
Epoch 2, Loss: 0.0495
Test set: Avg. loss: 0.0313, Accuracy: 6877/6949 (98.96%)
Test set: Avg. loss: 0.0313, Accuracy: 6877/6949 (98.96%)
Epoch 1, Loss: 0.1957
Test set: Avg. loss: 0.0530, Accuracy: 6833/6949

Etape : 100%|██████████| 8/8 [05:09<00:00, 38.68s/it]

Test set: Avg. loss: 0.0326, Accuracy: 6884/6949 (99.06%)


## Modèle AnB+

In [4]:
path = "./resultats/"


basemodel = torch.load("./models/baseA.torch")
loss_fn = nn.CrossEntropyLoss()


for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(10):
        newmodel = models.CNN()
        for i in range(1,j): # copie de certaines couches
            newmodel.conv[i] = basemodel.conv[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=2, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    with open(os.path.join(path, "ANB+.json"), 'a') as file:
        json.dump(res, file)

/tmp/ipykernel_3975/3452146040.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?, ?it/

Epoch 1, Loss: 1.0129
Test set: Avg. loss: 0.1020, Accuracy: 6708/6949 (96.53%)
Epoch 2, Loss: 0.0815
Test set: Avg. loss: 0.0459, Accuracy: 6844/6949 (98.49%)
Test set: Avg. loss: 0.0459, Accuracy: 6844/6949 (98.49%)
Epoch 1, Loss: 0.5291
Test set: Avg. loss: 0.0561, Accuracy: 6830/6949 (98.29%)
Epoch 2, Loss: 0.0660
Test set: Avg. loss: 0.0380, Accuracy: 6862/6949 (98.75%)
Test set: Avg. loss: 0.0380, Accuracy: 6862/6949 (98.75%)
Epoch 1, Loss: 0.8090
Test set: Avg. loss: 0.0833, Accuracy: 6790/6949 (97.71%)
Epoch 2, Loss: 0.0832
Test set: Avg. loss: 0.0534, Accuracy: 6832/6949 (98.32%)
Test set: Avg. loss: 0.0534, Accuracy: 6832/6949 (98.32%)
Epoch 1, Loss: 1.6313
Test set: Avg. loss: 1.6103, Accuracy: 1396/6949 (20.09%)
Epoch 2, Loss: 1.0486
Test set: Avg. loss: 0.1250, Accuracy: 6693/6949 (96.32%)
Test set: Avg. loss: 0.1250, Accuracy: 6693/6949 (96.32%)
Epoch 1, Loss: 0.7271
Test set: Avg. loss: 0.0909, Accuracy: 6761/6949 (97.29%)
Epoch 2, Loss: 0.0684
Test set: Avg. loss: 0.051

Etape :  12%|█▎        | 1/8 [00:38<04:26, 38.02s/it]

Test set: Avg. loss: 0.0486, Accuracy: 6851/6949 (98.59%)
Epoch 1, Loss: 1.0285
Test set: Avg. loss: 0.1064, Accuracy: 6732/6949 (96.88%)
Epoch 2, Loss: 0.0831
Test set: Avg. loss: 0.0662, Accuracy: 6813/6949 (98.04%)
Test set: Avg. loss: 0.0662, Accuracy: 6813/6949 (98.04%)
Epoch 1, Loss: 0.6658
Test set: Avg. loss: 0.0791, Accuracy: 6781/6949 (97.58%)
Epoch 2, Loss: 0.0681
Test set: Avg. loss: 0.0418, Accuracy: 6850/6949 (98.58%)
Test set: Avg. loss: 0.0418, Accuracy: 6850/6949 (98.58%)
Epoch 1, Loss: 0.5829
Test set: Avg. loss: 0.0577, Accuracy: 6824/6949 (98.20%)
Epoch 2, Loss: 0.0614
Test set: Avg. loss: 0.0521, Accuracy: 6846/6949 (98.52%)
Test set: Avg. loss: 0.0521, Accuracy: 6846/6949 (98.52%)
Epoch 1, Loss: 0.3003
Test set: Avg. loss: 0.0601, Accuracy: 6818/6949 (98.11%)
Epoch 2, Loss: 0.0599
Test set: Avg. loss: 0.0627, Accuracy: 6808/6949 (97.97%)
Test set: Avg. loss: 0.0627, Accuracy: 6808/6949 (97.97%)
Epoch 1, Loss: 0.5970
Test set: Avg. loss: 0.1021, Accuracy: 6732/6949

Etape :  25%|██▌       | 2/8 [01:16<03:50, 38.40s/it]

...

Test set: Avg. loss: 0.0374, Accuracy: 6871/6949 (98.88%)
Epoch 1, Loss: 0.3455
Test set: Avg. loss: 0.0517, Accuracy: 6835/6949 (98.36%)
Epoch 2, Loss: 0.0517
Test set: Avg. loss: 0.0330, Accuracy: 6880/6949 (99.01%)
Test set: Avg. loss: 0.0330, Accuracy: 6880/6949 (99.01%)
Epoch 1, Loss: 0.2293
Test set: Avg. loss: 0.0801, Accuracy: 6791/6949 (97.73%)
Epoch 2, Loss: 0.0549
Test set: Avg. loss: 0.0370, Accuracy: 6867/6949 (98.82%)
Test set: Avg. loss: 0.0370, Accuracy: 6867/6949 (98.82%)
Epoch 1, Loss: 0.2470
Test set: Avg. loss: 0.0682, Accuracy: 6796/6949 (97.80%)
Epoch 2, Loss: 0.0567
Test set: Avg. loss: 0.0433, Accuracy: 6859/6949 (98.70%)
Test set: Avg. loss: 0.0433, Accuracy: 6859/6949 (98.70%)
Epoch 1, Loss: 0.2073
Test set: Avg. loss: 0.0500, Accuracy: 6839/6949 (98.42%)
Epoch 2, Loss: 0.0527
Test set: Avg. loss: 0.0376, Accuracy: 6858/6949 (98.69%)
Test set: Avg. loss: 0.0376, Accuracy: 6858/6949 (98.69%)
Epoch 1, Loss: 0.2287
Test set: Avg. loss: 0.0599, Accuracy: 6822/6949

Etape :  88%|████████▊ | 7/8 [04:26<00:38, 38.12s/it]

Test set: Avg. loss: 0.0388, Accuracy: 6870/6949 (98.86%)
Epoch 1, Loss: 0.1814
Test set: Avg. loss: 0.0471, Accuracy: 6850/6949 (98.58%)
Epoch 2, Loss: 0.0515
Test set: Avg. loss: 0.0331, Accuracy: 6882/6949 (99.04%)
Test set: Avg. loss: 0.0331, Accuracy: 6882/6949 (99.04%)
Epoch 1, Loss: 0.1834
Test set: Avg. loss: 0.0539, Accuracy: 6816/6949 (98.09%)
Epoch 2, Loss: 0.0435
Test set: Avg. loss: 0.0298, Accuracy: 6886/6949 (99.09%)
Test set: Avg. loss: 0.0298, Accuracy: 6886/6949 (99.09%)
Epoch 1, Loss: 0.2019
Test set: Avg. loss: 0.0492, Accuracy: 6841/6949 (98.45%)
Epoch 2, Loss: 0.0471
Test set: Avg. loss: 0.0269, Accuracy: 6895/6949 (99.22%)
Test set: Avg. loss: 0.0269, Accuracy: 6895/6949 (99.22%)
Epoch 1, Loss: 0.1718
Test set: Avg. loss: 0.0414, Accuracy: 6854/6949 (98.63%)
Epoch 2, Loss: 0.0496
Test set: Avg. loss: 0.0779, Accuracy: 6779/6949 (97.55%)
Test set: Avg. loss: 0.0779, Accuracy: 6779/6949 (97.55%)
Epoch 1, Loss: 0.1803
Test set: Avg. loss: 0.0485, Accuracy: 6845/6949

Etape : 100%|██████████| 8/8 [05:04<00:00, 38.02s/it]

Test set: Avg. loss: 0.0338, Accuracy: 6866/6949 (98.81%)
